In [1]:
# pip install tabulate

In [2]:
import requests
import json
import pandas as pd
import datetime
import time
import os
import queries # ./queries.py
# https://jerrynsh.com/how-i-sync-daily-leetcoding-challenge-to-todoist/
# https://towardsdatascience.com/connecting-to-a-graphql-api-using-python-246dda927840

USERNAME = 'jhojin7'

success queries.py


In [3]:
def get_recents(username):
    base_url = 'https://leetcode.com/graphql/'
    url = base_url
    print(url)
    new_query = """ 
    query {
        recentSubmissionList(username:"%s") { 
            title   
            titleSlug   
            timestamp  
            statusDisplay   
            lang    
            __typename
        }
    }
    """ %(username)
    # https://towardsdatascience.com/connecting-to-a-graphql-api-using-python-246dda927840
    # return requests.post(url,headers=headers,data=new_query)
    result = requests.post(url,json={'query':new_query})
    print(result.status_code)
    json_dump = json.loads(result.text)
    # print(json_dump)
    recents = json_dump['data']['recentSubmissionList']
    recents_df = pd.DataFrame(recents)
    # print(recents_df)
    return recents_df


In [4]:

def update_recents():
    recents_df = get_recents(USERNAME)
    recents_df.head()
    # filter and rename only the content I need
    recents_df = recents_df.filter(['titleSlug','statusDisplay','timestamp'])
    recents_df = recents_df.rename(columns={'titleSlug':'Title', 'statusDisplay':'Status','timestamp':'Datetime'})
    # change datetime format from epoch-seconds to iso8601
    for t in recents_df['Datetime']:
        # stop if datetime is already string format
        if t.isnumeric() == False: break
        dtext = time.localtime(int(t))
        # iso8601 datetime format string
        new_t = "%d-%02d-%02d %02d:%02d:%02d" %(dtext.tm_year,dtext.tm_mon,dtext.tm_mday,dtext.tm_hour,dtext.tm_min,dtext.tm_sec)
        recents_df['Datetime'] = new_t

    # output content
    # print(f"{username}'s Most Recent Submissions:")
    # print(recents_df)

    # write to file
    today = time.time()
    now = time.strftime("%Y-%m-%d %H:%M:%S")
    output = f"""# {USERNAME}'s Most Recent Submissions:\nUpdated: {now}\n"""

    with open("leetcode-recents.md",'w') as f:
        # output text
        f.writelines(output)
        # 10 top rows to markdown table
        f.writelines(str(recents_df.head(10).to_markdown()))
    f.close()
update_recents()

https://leetcode.com/graphql/
200


In [5]:
# query_dailyCodingChallenge = """
# query dailyCodingQuestionRecords(year: %s, month: %s) {
#   dailyCodingChallengeV2(year: %s, month: %s) {
#     challenges {
#       date
#       userStatus
#       link
#       question {
#         questionFrontendId
#         title
#         titleSlug
#       }
#     }
#     weeklyChallenges {
#       date
#       userStatus
#       link
#       question {
#         questionFrontendId
#         title
#         titleSlug
#       }
#     }
#   }
# }
# """ %(yyyy, mm, yyyy, mm)

def f_write(fname,lines):
    with open(fname,'w') as f:
        f.writelines(lines)
    f.close()

yyyy = '2022'
mm = '3'

def get_dailyChallenges(query):
  base_url = 'https://leetcode.com/graphql/'
  url = base_url
  # https://towardsdatascience.com/connecting-to-a-graphql-api-using-python-246dda927840
  # return requests.post(url,headers=headers,data=new_query)
  result = requests.post(url,json={'query':query%(yyyy,mm)})
  print(url, result.status_code)
  json_dump = json.loads(result.text)
  # print(json_dump)
  dailyChallenges = json_dump['data']['dailyCodingChallengeV2']['challenges']
  # print(dailyChallenges)
  daily_df = pd.DataFrame(dailyChallenges)
  f_write('leetcode_daily_%s-%0s.md'%(yyyy,mm),daily_df.to_markdown())
  return daily_df
  # print(daily_df)

daily_df = get_dailyChallenges(queries.QUERY_DAILYCODINGCHALLENGE)
daily_df

https://leetcode.com/graphql/ 200


,date,userStatus,link,question
0,2022-03-01,NotStart,/problems/counting-bits/,"{'questionFrontendId': '338', 'title': 'Counti..."
1,2022-03-02,NotStart,/problems/is-subsequence/,"{'questionFrontendId': '392', 'title': 'Is Sub..."
2,2022-03-03,NotStart,/problems/arithmetic-slices/,"{'questionFrontendId': '413', 'title': 'Arithm..."
3,2022-03-04,NotStart,/problems/champagne-tower/,"{'questionFrontendId': '799', 'title': 'Champa..."
4,2022-03-05,NotStart,/problems/delete-and-earn/,"{'questionFrontendId': '740', 'title': 'Delete..."


In [6]:
# data.dailyCodingChallengeV2.challenges[1].question.questionFrontendId
daily_df['question']
titleSlug = daily_df['question'][0]['titleSlug']
qId = daily_df['question'][0]['questionFrontendId']
print(qId, titleSlug)


338 counting-bits


In [7]:
# for q in recents_df.itertuples():
#     # print(q)
#     titleSlug = q.titleSlug
#     questionLink = f"https://leetcode.com/problems/{titleSlug}"
#     submissionLink = f"https://"
#     print(questionLink)
#     # print(submissionLink)

In [8]:
# print("{%Y}-{%m}-{%d}T{%H}:{%M}:{%S}".format(dtext))
# datetime.datetime(dtext.tm_year,dtext.tm_mon,dtext.tm_mday,dtext.tm_hour,)

In [9]:
# probably needs login auth to view the actual code submissions??...?
def get_submissions(query,questionSlug):
    base_url = 'https://leetcode.com/graphql/'
    url = base_url
    headers = {"Authorization": "Bear token"}
    result = requests.post(url, 
        json={'query': query%(questionSlug)})#, headers=headers)
    print(result.status_code) # 499= maybe CSRF?
    # print(result.content)
    json_dump = json.loads(result.text)
    print(json_dump)
    json_data = json_dump['data']['submissionList']
    json_df = pd.DataFrame(json_data)
    print(json_df)
    return json_df

# questionSlug = "binay-search"
# submits_df = get_submissions(queries.QUERY_SUBMISSIONS,questionSlug)
# print(submits_df)


# recents_df
# for q in recents_df.itertuples():
#     titleSlug = q.titleSlug
#     print(titleSlug)
#     submits_df = get_submissions(titleSlug)
#     print(submits_df)

